Build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0)
dfToronto=df[0]
dfToronto.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Ignore the cells without a Borough:

In [21]:
dfToronto = dfToronto[dfToronto.Borough!= 'Not assigned'].reset_index(drop=True)

Grouping different neighborhoods and separate them with comma

In [24]:
dfToronto_grouped=dfToronto.groupby(['Postal Code','Borough'], as_index=False).agg(lambda x: ','.join(x))

Finally, assign empty neighborhood:

In [31]:
dfToronto_grouped.loc[dfToronto_grouped.Neighbourhood== "Not assigned", 'Neighbourhood'] = dfToronto_grouped.loc[dfToronto_grouped.Neighbourhood== "Not assigned", 'Borough']
dfToronto_grouped.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [37]:
dfToronto_grouped.shape

(103, 3)

Get dataframe from csv file that has the geographical coordinates of each postal code:

In [38]:
coordinates=pd.read_csv('https://cocl.us/Geospatial_data')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Append each latitude and longitude to the PostcalCode. Use a simple inner join with the csv file.

In [40]:
neighborhood = pd.merge(dfToronto_grouped, coordinates, on='Postal Code', how='inner')
neighborhood.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
